In [1]:
import os
import json
import pandas as pd
import json

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
KEY=os.getenv("OPENAI_API_KEY")

In [5]:
llm=ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature = 0.5)

c:\Projects\OpenAI\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "4":
        {
            "three_marks_questions": "three_marks_questions",
            "type": "short_answer",
            "question": "Three marks question here"
        },
    "5":
        {
            "three_marks_questions": "three_marks_questions",
            "type": "short_answer",
            "question": "Three marks question here"
        },
    "6": 
        {
            "five_marks_questions": "five_marks_questions",
            "type": "long_answer",
            "question": "Five marks question here"
        },
    "7":
        {
            "five_marks_questions": "five_marks_questions",
            "type": "long_answer",
            "question": "Five marks question here"
        }
}

In [26]:
TEMPLATE="""
Text: {text}
You are an expert in creating MCQs and detailed answer questions. Given the above text, your task is to create a quiz comprising {mcq_number} multiple choice questions (MCQs), {three_marks_number} questions worth 3 marks, and {five_marks_number} questions worth 5 marks for {subject} students. Maintain a {tone} tone throughout the quiz. 
Ensure the questions are unique, relevant to the provided text, and that they accurately assess the students' understanding of the subject matter.
Adhere to the following format in your RESPONSE_JSON below as a guide. Create {mcq_number} MCQs, {three_marks_number} questions for 3 marks, and {five_marks_number} questions for 5 marks, ensuring a comprehensive evaluation.

{response_json}
"""

In [27]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "mcq_number", "three_marks_number", "five_marks_number", "subject", "tone", "response_json"],
    template=TEMPLATE
)

In [28]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [29]:
TEMPLATE2 = """
You are an expert English grammarian and writer. Given a comprehensive quiz for {subject} students, including multiple choice questions (MCQs), questions worth 3 marks, and questions worth 5 marks, your task is to evaluate the overall complexity of the quiz. Provide a succinct complexity analysis, not exceeding 50 words. Assess whether the quiz aligns with the cognitive and analytical abilities of the intended student audience. 

If you find any aspects of the quiz not up to par—whether it be the content, cognitive demands, or tone—identify these issues specifically. Suggest necessary modifications to ensure the quiz questions are appropriately challenging and engaging, and adjust the tone to perfectly match the students' abilities.

Content:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [30]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [31]:
review_chain = LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [32]:
generate_evaluate_chain = SequentialChain(
    chains=[quiz_chain, review_chain],
    input_variables=["text", "mcq_number", "three_marks_number", "five_marks_number", "subject", "tone", "response_json"],
    output_variables=["quiz", "review"],
    verbose=True
)

In [33]:
file_path=r"C:\Projects\OpenAI\mcqgen\data.txt"

In [34]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [35]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [36]:
json.dumps(RESPONSE_JSON)

'{"mcqs": [{"question_id": "1", "type": "mcq", "question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}, {"question_id": "2", "type": "mcq", "question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}, {"question_id": "3", "type": "mcq", "question": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct_answer": "correct answer"}], "three_marks_questions": [{"question_id": "4", "type": "short_answer", "question": "Three marks question here"}, {"question_id": "5", "type": "short_answer", "question": "Three marks question here"}], "five_marks_questions": [{"question_id": "6", "type": "long_answer", "question": "Five marks question here"}, {"question_id": "7", "type": "long_answer", "question": "Fi

In [50]:
MCQ_NUMBER=5 
THREE_MARKS_NUMBER = 5
FIVE_MARKS_NUMBER = 5
SUBJECT="biology"
TONE="simple"

In [51]:
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
        {
            "text": TEXT,
            "mcq_number": MCQ_NUMBER,
            "three_marks_number": THREE_MARKS_NUMBER,
            "five_marks_number": FIVE_MARKS_NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defin

In [52]:

print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1731
Prompt Tokens:1308
Completion Tokens:423
Total Cost:0.002808


In [53]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [54]:
quiz_data_json = response['quiz']

In [55]:
quiz_data = json.loads(quiz_data_json)

In [56]:
questions = []

In [57]:
for mcq in quiz_data['mcqs']:
    questions.append({
        'Type': 'MCQ',
        'Question ID': mcq['question_id'],
        'Question': mcq['question'],
        'Options': mcq['options'],
        'Correct Answer': mcq['correct_answer']
    })

for three_marks in quiz_data['three_marks_questions']:
    questions.append({
        'Type': 'Three Marks',
        'Question ID': three_marks['question_id'],
        'Question': three_marks['question'],
        'Options': None,
        'Correct Answer': None  # Assuming no correct answer field for these
    })

# Process five marks questions
for five_marks in quiz_data['five_marks_questions']:
    questions.append({
        'Type': 'Five Marks',
        'Question ID': five_marks['question_id'],
        'Question': five_marks['question'],
        'Options': None,
        'Correct Answer': None  # Assuming no correct answer field for these
    })

In [59]:
df_questions = pd.DataFrame(questions)

# Print the DataFrame to see the output
print(df_questions)

# Save the DataFrame to an Excel file
df_questions.to_excel('quiz_questions.xlsx', index=False)

          Type Question ID                                           Question  \
0          MCQ           1  What is the unifying theme in biology that exp...   
1          MCQ           2  Which of the following is NOT a major theme in...   
2          MCQ           3  What is the primary function of genes in organ...   
3  Three Marks           4  Explain the significance of energy processing ...   
4  Three Marks           5  Describe the levels of organization at which b...   
5   Five Marks           6  Discuss the role of biodiversity in maintainin...   
6   Five Marks           7  Explain how biologists use the scientific meth...   

                                             Options Correct Answer  
0  {'a': 'Evolution', 'b': 'Energy processing', '...              a  
1  {'a': 'Energy processing', 'b': 'Evolution', '...              c  
2  {'a': 'Regulating internal environments', 'b':...              c  
3                                               None           None  
4

In [ ]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_23_2024_18_31_36'